In [1]:
import pyodbc
import boto3
from botocore.exceptions import NoCredentialsError
import os
import pandas as pd

In [2]:
# Database Connection Configuration
server = 'KATANA'
database = 'Chinook'
username = 'sa'
password = ''
driver = '{SQL Server}'

In [3]:
# MinIO Configuration
minio_endpoint = 'http://localhost:9000'
minio_access_key = 'minioadmin'
minio_secret_key = 'minioadmin'
bucket_name = 'raw'

In [4]:
# Paths for Temporary CSV Files
output_path = "C:/ChinookData/"
os.makedirs(output_path, exist_ok=True)  

In [5]:
def get_table_names():
    try:
        print("Fetching table names from SQL Server...")
        conn = pyodbc.connect(
            f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
        )
        query = """
        SELECT TABLE_NAME 
        FROM INFORMATION_SCHEMA.TABLES 
        WHERE TABLE_TYPE = 'BASE TABLE';
        """
        table_names = pd.read_sql(query, conn)['TABLE_NAME'].tolist()
        conn.close()
        print(f"Found tables: {table_names}")
        return table_names
    except Exception as e:
        print(f"Error fetching table names: {e}")
        return []

In [6]:
def extract_raw_data(table_names):
    try:
        print("Connecting to SQL Server for data extraction...")
        conn = pyodbc.connect(
            f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
        )
        for table in table_names:
            print(f"Extracting data for table: {table}...")
            query = f"SELECT * FROM [{table}]"
            file_path = f"{output_path}{table}.csv"
            df = pd.read_sql(query, conn)
            df.to_csv(file_path, index=False)
            print(f"Saved data to: {file_path}")
        conn.close()
        print("Data extraction complete.")
    except Exception as e:
        print(f"Error during data extraction: {e}")


In [7]:
# Function to Load Files into MinIO (Raw Zone)
def load_to_minio_raw(table_names):
    try:
        print("Connecting to MinIO for file upload...")
        s3_client = boto3.client('s3',
                                 endpoint_url=minio_endpoint,
                                 aws_access_key_id=minio_access_key,
                                 aws_secret_access_key=minio_secret_key)
        for table in table_names:
            file_path = f"{output_path}{table}.csv"
            if os.path.exists(file_path):
                print(f"Uploading {table}.csv to MinIO Raw Zone...")
                s3_client.upload_file(file_path, bucket_name, f"{table}.csv")
                print(f"Uploaded {table}.csv to bucket '{bucket_name}'")
            else:
                print(f"File {file_path} does not exist. Skipping upload.")
        print("File upload complete.")
    except NoCredentialsError:
        print("Credentials not available.")
    except Exception as e:
        print(f"Error during file upload: {e}")


In [8]:
# Main Workflow
if __name__ == "__main__":
    print("Starting ELT Extract and Load Phase...")
    table_names = get_table_names()  # Step 1: Get all table names dynamically
    if table_names:
        extract_raw_data(table_names)  # Step 2: Extract data for all tables
        load_to_minio_raw(table_names)  # Step 3: Upload files to MinIO
    else:
        print("No tables found in the database. Exiting.")
    print("ELT Extract and Load Phase Completed Successfully!")

Starting ELT Extract and Load Phase...
Fetching table names from SQL Server...


C:\Users\adnan\AppData\Local\Temp\ipykernel_24548\1008654608.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_names = pd.read_sql(query, conn)['TABLE_NAME'].tolist()


Found tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track', 'sysdiagrams']
Connecting to SQL Server for data extraction...
Extracting data for table: Album...
Saved data to: C:/ChinookData/Album.csv
Extracting data for table: Artist...
Saved data to: C:/ChinookData/Artist.csv
Extracting data for table: Customer...
Saved data to: C:/ChinookData/Customer.csv
Extracting data for table: Employee...
Saved data to: C:/ChinookData/Employee.csv
Extracting data for table: Genre...
Saved data to: C:/ChinookData/Genre.csv
Extracting data for table: Invoice...
Saved data to: C:/ChinookData/Invoice.csv
Extracting data for table: InvoiceLine...
Saved data to: C:/ChinookData/InvoiceLine.csv
Extracting data for table: MediaType...
Saved data to: C:/ChinookData/MediaType.csv
Extracting data for table: Playlist...
Saved data to: C:/ChinookData/Playlist.csv
Extracting data for table: PlaylistTrack...
Saved data to: C:/Chi